In [96]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [97]:
import pandas as pd
import numpy as np
#NLP Environment setup 
from collections import Counter 

#Plotting 
import squarify
import matplotlib.pyplot as plt 
import seaborn as sns 

#NLP Libraries 
import re
from nltk.stem import PorterStemmer
import spacy
from spacy.tokenizer import Tokenizer#NLP Environment setup 
from collections import Counter 

#Plotting 
import squarify
import matplotlib.pyplot as plt 
import seaborn as sns 

#NLP Libraries 
import re
from nltk.stem import PorterStemmer
import spacy
from spacy.tokenizer import Tokenizer


In [98]:
import gensim
import os
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_numeric, stem_text
from gensim.parsing.preprocessing import strip_multiple_whitespaces, strip_non_alphanum, remove_stopwords, strip_short

In [99]:
df = pd.read_csv('/Users/noahpovis/Desktop/Buildweek22/Build Week 2 /DX.csv')


In [100]:
train, val, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])


In [101]:
train.shape

(16278, 18)

In [102]:
val.shape

(5426, 18)

In [103]:
test.shape

(5427, 18)

In [104]:
def clean(i):
    i = i.copy()
    i.drop_duplicates(subset=['Organization Name','Funding Type'], inplace = True)
    i.drop_duplicates(keep='first', inplace= True)
    i['Raised Series A'] = np.where(i['Organization Name'].duplicated(), 1, 0)
    i.drop_duplicates(subset='Organization Name', keep='last', inplace= True)
    i['Money Raised Currency (in USD)'] = pd.to_numeric(i['Money Raised Currency (in USD)'], errors='coerce')
    i['Total Funding Amount Currency (in USD)'] = pd.to_numeric(i['Total Funding Amount Currency (in USD)'], errors='coerce')
    i['Announced Date'] = pd.to_datetime(i['Announced Date'], errors='coerce')
    i['Total Funding Amount Currency (in USD)'] = pd.to_numeric(i['Total Funding Amount Currency (in USD)'], errors='coerce')
    i  = i[i['Funding Type'] != 'Series A'] 
    return i 



In [105]:
train = clean(train)
val = clean(val)
test = clean(test)


In [106]:
train['Raised Series A'].value_counts()

0    12342
1      180
Name: Raised Series A, dtype: int64

In [107]:
train.shape , val.shape, test.shape

((12522, 19), (4546, 19), (4547, 19))

In [108]:
train['Investor Names'].isnull().sum()

3660

In [109]:
def split_columns(text):
    text = text.copy()
    # new data frame with split value columns 
    new = text["Organization Industries"].str.split(",", n = 7, expand = True) 
    # making separate first name column from new data frame 
    text["Primary Industry"]= new[0] 
    # making separate last name column from new data frame 
    text["Sub_Ind"]= new[1] 
    text["Sub_Ind2"]= new[2]
    text["Sub_Ind3"]= new[3]
    text["Sub_Ind4"]= new[4]
    text["Sub_Ind5"]= new[5]
    text["Sub_Ind6"]= new[6]
    text["Sub_Ind7"]= new[7]
    # Dropping old Name columns 
    text.drop(columns =["Organization Industries"], inplace = True)

    #same thing but for investor column 
    new1 = text["Investor Names"].str.split(",", n = 11, expand = True) 
    # making separate first name column from new data frame 
    text["Investor1"]= new1[0] 
    # making separate last name column from new data frame 
    text["Investor1"]= new1[1] 
    text["Investor2"]= new1[2]
    text["Investor3"]= new1[3]
    text["Investor4"]= new1[4]
    text["Investor5"]= new1[5]
    text["Investor6"]= new1[6]
    text["Investor7"]= new1[7]
    text["Investor8"]= new1[8]
    text["Investor9"]= new1[9]
    text["Investor10"]= new1[10]
    text["Investor11"]= new1[11]
    # Dropping old Name columns 
    text.drop(columns =["Investor Names"], inplace = True)
    text.drop(columns =["Lead Investors"], inplace = True)
    text.drop(columns =["Number of Investors"], inplace = True)
    text = text[text['Money Raised'].notna()]
    text = text[text['Organization Location'].notna()]
    text = text[text['Primary Industry'].notna()]
    text = text[text['Money Raised Currency (in USD)'].notna()]
    



    return text 



In [110]:
train = split_columns(train)
val = split_columns(val)
test = split_columns(test)


In [111]:
train.columns

Index(['Transaction Name', 'Transaction Name URL', 'Organization Name',
       'Organization Name URL', 'Funding Type', 'Money Raised',
       'Money Raised Currency', 'Money Raised Currency (in USD)',
       'Announced Date', 'Total Funding Amount',
       'Total Funding Amount Currency',
       'Total Funding Amount Currency (in USD)', 'Organization Description',
       'Organization Location', 'Raised Series A', 'Primary Industry',
       'Sub_Ind', 'Sub_Ind2', 'Sub_Ind3', 'Sub_Ind4', 'Sub_Ind5', 'Sub_Ind6',
       'Sub_Ind7', 'Investor1', 'Investor2', 'Investor3', 'Investor4',
       'Investor5', 'Investor6', 'Investor7', 'Investor8', 'Investor9',
       'Investor10', 'Investor11'],
      dtype='object')

In [112]:
train.isnull().sum()

Transaction Name                             0
Transaction Name URL                         0
Organization Name                            0
Organization Name URL                        0
Funding Type                                 0
Money Raised                                 0
Money Raised Currency                        0
Money Raised Currency (in USD)               0
Announced Date                               0
Total Funding Amount                         0
Total Funding Amount Currency                0
Total Funding Amount Currency (in USD)       0
Organization Description                     0
Organization Location                        0
Raised Series A                              0
Primary Industry                             0
Sub_Ind                                   1068
Sub_Ind2                                  2556
Sub_Ind3                                  4677
Sub_Ind4                                  6013
Sub_Ind5                                  6840
Sub_Ind6     

In [113]:
train['Raised Series A'].value_counts()

0    7806
1     127
Name: Raised Series A, dtype: int64

In [114]:
train['Money Raised Currency (in USD)'].median()

750000.0

In [115]:
# Defining 'preprocessing_filter' function
def preprocessing_filter(text):
    '''
    Cleans text for font size, whitespace,
    removing stop words, and stripping non-alpha 
    numeric characters
    '''
    # Custom Preprocessing Filter
    custom_filters = [lambda x: x.lower(),          # Lowercase
                      strip_multiple_whitespaces,   # Remove numbers
                      remove_stopwords,             # Remove stopwords
                      strip_non_alphanum,           # Remove non-alpha numeric characters
                     ]
    
    
    # TEST / CHECK: Preprocess with 'custom_filters'
    processed_text = preprocess_string(text, custom_filters)
    
    # Returning 'processed_text'
    return processed_text

In [116]:
def function_apply(k):
    k["tokenized_Description"] = k["Organization Description"].apply(preprocessing_filter)
    k.drop(columns =["Organization Description"], inplace = True)
    print(k.shape)

    return k 

In [117]:
train = function_apply(train)
val = function_apply(val)
test = function_apply(test)

(7933, 34)
(2933, 34)
(2914, 34)


In [118]:
# Arrange data into X features matrix and y target vector
target = 'Raised Series A'
X_train = train.drop(columns=[target,'tokenized_Description','Announced Date'])
y_train = train[target]
X_val = val.drop(columns=[target,'tokenized_Description','Announced Date'])
y_val = val[target]
X_test = test.drop(columns=[target,'tokenized_Description','Announced Date'])
y_test = test[target]

In [119]:
X_train.shape , y_train.shape

((7933, 31), (7933,))

In [137]:
val['Raised Series A'].value_counts()

0    2915
1      18
Name: Raised Series A, dtype: int64

In [127]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import recall_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_curve

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    XGBClassifier(n_estimators=100,
                  random_state=42,
                  n_jobs=-1          
)
)

###scores = cross_val_score(pipeline, X_train, y_train, cv=10, 
                         ##scoring = 'average_precision')

###scores_val = cross_val_score(pipeline, X_val, y_val, cv=10, 
                         ##scoring = 'average_precision')'''


pipe = pipeline.fit(X_train, y_train);




In [130]:
xgb_val = pipe.predict(X_val)
print("XGBoost Classifier val Recall Accuracy:", recall_score(y_val, xgb_val))
print("XGBoost Classifier val Accuracy Score:", accuracy_score(y_val, xgb_val))
print("XGBoost Classifier val Precision Accuracy:", precision_score(y_val, xgb_val))

XGBoost Classifier val Recall Accuracy: 0.0
XGBoost Classifier val Accuracy Score: 0.9911353562904875
XGBoost Classifier val Precision Accuracy: 0.0


In [138]:
# when i ran predict(on X_val, it predicted that 10 of the 18 'Yes' values incorrectly. This could explain why the precision and recall accuracy is so low. 
unique_elements, counts_elements = np.unique(xgb_val, return_counts=True)
print(np.asarray((unique_elements, counts_elements)))

[[   0    1]
 [2925    8]]


In [159]:
#make another 
target = 'Raised Series A'
X_train = train.drop(columns=[target,'tokenized_Description','Announced Date'])
y_train = train[target]
X_val = val.drop(columns=[target,'tokenized_Description','Announced Date'])
y_val = val[target]
X_test = test.drop(columns=[target,'tokenized_Description','Announced Date'])
y_test = test[target]

In [144]:
#install imbalance package
pip install -U imbalanced-learn

     |████████████████████████████████| 167 kB 839 kB/s 
Note: you may need to restart the kernel to use updated packages.


In [160]:
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state = 2) 
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

ValueError: could not convert string to float: 'Seed Round - Fashion Kanvas'

In [123]:
#def corpora(d):
    #'Dictionary' Representation of all the words in our corpus
    #id2word = corpora.Dictionary(d["tokenized_Description"])
    #Removing extreme values from the dataset
    #id2word.filter_extremes(no_below=5, no_above=0.95)
    #bag of words(bow) representation of our corpus
    #corpus = [id2word.doc2bow(text) for text in d["tokenized_Description"]]
    #return d 

In [124]:
#train = corpora(train)
#val = corpora(val)
#test = corpora(test)





In [125]:
# precision, recall and F1
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
y_train = np.array([number[0] for number in lb.fit_transform(y_train)])

recall = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='recall')
print('Recall', np.mean(recall), recall)
precision = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='precision')
print('Precision', np.mean(precision), precision)
f1 = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='f1')
print('F1', np.mean(f1), f1)

Recall 0.20800000000000002 [0.   0.   0.   0.12 0.92]
Precision 0.008545885467614697 [0.         0.         0.         0.01038062 0.0323488 ]
F1 0.016321656050955417 [0.         0.         0.         0.01910828 0.0625    ]
